## The Thanksgiving data**

This data came from Five-Thirty-Eight's data repository. I give them full credit for the collection of the data.   https://github.com/fivethirtyeight/data/tree/master/thanksgiving-2015)

\*\* *This data is no way representitive of the total US population.  Five-thirty-eight polled 1058 respondants on Nov 17th 2015.  Furthermore, due to small sample size, data is unlikely to be statistically significant.  This project was for learning purposes (NumPy, Pandas, Jupyter).  *


## Goals

** Some questions that we set out to answer during this project!**

* What is the most popular dish at Thanksgiving dinner?  What is the most popular meal?
* Tofurkey is a main thanksgiving dish comprised of both Tofu and Turkey.  What are the main side dishes that tofurkey eaters have during the holiday?
* Is travel correlated with age or income?
* Is celebrating Friendsgiving correlated with age? 

## Motivations 

* To learn Pandas and Numpy;  two cool python libraries used for Data Analysis.  

**Numpy:**
* Makes it easy to create and manupulate data stored in multi-dimensional arrays.

**Pandas:**
* Built on top of numpy
* Flexibility to hold data of different types!
* Flexibility to deal with missing values.

In [1]:


import pandas as pd  

data = pd.read_csv("thanksgiving.csv", 
                   encoding="Latin-1")

## Remove responses from people who are not celebrating the holiday:

In [2]:
is_american = 'Do you celebrate Thanksgiving?'
print(data[is_american].value_counts())

Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64


In [3]:
america_bool = data[is_american] == "Yes"
americans = data[america_bool]


### What are is the most common meal at Thanksgiving?

In [4]:
main_dish = "What is typically the main dish at your Thanksgiving dinner?"
main_cooked = 'How is the main dish typically cooked?'
stuffing = 'What kind of stuffing/dressing do you typically have?'
cran = 'What type of cranberry saucedo you typically have?'

main_dishes = americans[[main_cooked, main_dish, stuffing, cran]].copy()
main_dishes.mode()

,How is the main dish typically cooked?,What is typically the main dish at your Thanksgiving dinner?,What kind of stuffing/dressing do you typically have?,What type of cranberry saucedo you typically have?
0,Baked,Turkey,Bread-based,Canned


### Baked Turkey with stuffing was by far the most common main dish in Folk's Thanksgiving Dinner

### Next: Find most common 1) Side Dish 2) Pie, 3) Dessert

In [5]:
import re
import math
# create a small MaxSideDish module that calculates most popular side dishes based on a food selector.
# We can use this module for side dishes, desserts, and pie!

class MaxSideDish:
    def __init__(self, col_name, df):
        self.col_name = col_name
        self.df = df
        
        # Which columns from the dataframe (df) match the col_name in question.  
        # (i.e. with "side dishes" as col_name, match all columns that mention side dishes)
        #
        self.filter_names()
        
        # create new dataframe, col_dishes, that contain all columns that match our col_name attribute
        self.col_dishes = df[self.food_names].copy() 
        
        
        # Use regex to shorten column names
        self.shorten_list()
        
        # Find count of food per column
        self.aggregate_food_counts()
        
        #Find food with max count
        self.most_common_side = max(self.most_common_sides, key=self.most_common_sides.get)
    
    def filter_names(self):
        regex = re.compile(self.col_name).search
        self.food_names = [ name for name in self.df for m in (regex(name),) if m ]
    
    def shorten_list(self):
        
        filter = re.compile('(?<= - ).*').search
        
        self.short_names = [ m.group(0) for l in self.food_names for m in (filter(l),) if m]
        self.col_dishes.columns = self.short_names
    
        
    def aggregate_food_counts(self):
        most_common_sides = {}
        for food in self.col_dishes.columns:
            count_food = self.col_dishes[food].value_counts()
            max_food_count = count_food.max()
            if not math.isnan(max_food_count):
                most_common_sides[food] = max_food_count

        self.most_common_sides = most_common_sides

In [6]:
side_dishes = MaxSideDish("side dishes", americans)

side_dishes.most_common_side

'Mashed potatoes'

In [7]:
pie = MaxSideDish("type of pie", americans)
pie.most_common_side

'Pumpkin'

In [8]:
desserts = MaxSideDish("dessert", americans)
desserts.most_common_side

'None'

### Dessert has 'None' as most common. 
##### Let's take a look at the breakdown for dessert to get a larger picture.

In [9]:
desserts.most_common_sides

{'Apple cobbler': 110,
 'Blondies': 16,
 'Brownies': 128,
 'Carrot cake': 72,
 'Cheesecake': 191,
 'Cookies': 204,
 'Fudge': 43,
 'Ice cream': 266,
 'None': 295,
 'Other (please specify)': 134,
 'Other (please specify).1': 13,
 'Peach cobbler': 103}

##### Ice cream is the second most common dessert, behind None'

In [10]:

tofurkey_eaters_bool = americans[main_dish] == "Tofurkey"
# Who ate the tofurkey?
tofurkeyers = americans[tofurkey_eaters_bool]

# What were the most popular side dish, pie, and dessert of tofurkey eaters?
tofu_and_pie = MaxSideDish("type of pie", tofurkeyers)
tofu_and_dessert = MaxSideDish("dessert", tofurkeyers)
tofu_and_side_dish = MaxSideDish("side dishes", tofurkeyers)

print("There were {} tofurkey eaters\nTheir favorite side dish was {}\nTheir favorite pie was {}.\nTheir favorite dessert was {}.\n".format(len(tofurkeyers.values),
                tofu_and_pie.most_common_side,
                tofu_and_dessert.most_common_side,
                tofu_and_side_dish.most_common_side))


There were 20 tofurkey eaters
Their favorite side dish was Pumpkin
Their favorite pie was Ice cream.
Their favorite dessert was Mashed potatoes.



In [11]:
def convert_age(n):
    if pd.isnull(n):
        return None
    
    age = str(n).split(" ")[0]
    age = age.replace("+","")
    return int(age)

In [12]:
americans["int_age"] = americans["Age"].apply(convert_age)
print(americans["int_age"].describe())

count    947.000000
mean      40.089757
std       15.352014
min       18.000000
25%             NaN
50%             NaN
75%             NaN
max       60.000000
Name: int_age, dtype: float64


# Actual ages not used
### Age responses were converted from ranges to numbers for simplicity.

| Original Range  | Age used for analysis  |
|-----------------|------------------------|
| 18 - 29 | 18  | 
| 30 - 44  | 30  | 
| 45 - 59  |  45 |
| 60+  | 60  |
 


In [13]:
def convert_income(n): # i.e. $10,000
    if pd.isnull(n):
        return None 
    
    income = n.split(" ")[0]
    
    if (income == "Prefer"):
        return None
    
    income = income.replace("$", "")
    income = income.replace(",", "")
    
    return int(income) #==> 10000

In [14]:
income_q = "How much total combined money did all members of your HOUSEHOLD earn last year?"

americans["int_income"] = americans[income_q].apply(convert_income)


# Not a true depiction of incomes.  
### Income responses were converted from ranges to numbers for simplicity.


| Original Income Range  | Income used for analysis  |
|------------------------|---------------------------|
|  \$0                    |  \$0                        | 
|  \$10,000 - \$24,000    |  \$10,000                    | 
|  \$25,000 - \$49,000     |  \$25,000                    |
|  \$50,000 - \$74,999     |  \$50,000                    |
|  \$75,000 - \$99,999     |  \$75,000                    |
|  \$100,000 - \$124,999   |  \$100,000                   |
|  \$125,000 - \$149,999   |  \$125,000                   |
|  \$150,000 - \$174,999   |  \$150,000                   |
|  \$175,000 - \$199,999   |  \$175,000                   |
|  \$200,000+             |  \$200,000                   |

### Seemingly larger number of responses for the final category  (\$200,000+) is just because it covers the biggest income range.

| Income  | Respondents  |
|---------|--------------|
|  \$0       |       52  |
|  \$10,000   |       60  |
|  \$25,000   |      166  |
|  \$50,000   |      127  |
|  \$75,000   |      127  |
|  \$100,000  |      109  |
|  \$125,000  |       48  |
|  \$150,000  |       38  |
|  \$175,000  |       26  |
|  \$200,000  |       76  |

## Calculate breakdown of travel by Income
##### Test the hypothesis that low income earners are more likely to travel due to them travelling to their parents house


###### First create a pivot table broken down by Travel and Income

In [15]:
travel_q = "How far will you travel for Thanksgiving?"
index_id = "RespondentID"
income = "int_income"
no_travel = "Thanksgiving is happening at my home--I won't travel at all"


travel_by_income = pd.pivot_table(americans, 
                                  values=[index_id], 
                                  index=[travel_q],
                                  columns=[income],
                                  aggfunc=len, margins=True)

# next calculate percentabes by value 

travel_by_income_as_pct = travel_by_income.div(travel_by_income.iloc[-1,:], axis=1)
pct_no_travel = travel_by_income_as_pct.loc[no_travel]

In [16]:
pct_travel = (1 - pct_no_travel)
print(pct_travel)

              int_income
RespondentID  0.0           0.596154
              10000.0       0.583333
              25000.0       0.638554
              50000.0       0.677165
              75000.0       0.543307
              100000.0      0.541284
              125000.0      0.458333
              150000.0      0.552632
              175000.0      0.576923
              200000.0      0.500000
              All           0.581423
Name: Thanksgiving is happening at my home--I won't travel at all, dtype: float64


| Income  | Percent of Income Group who Travel  |
|---------|-------------------------------------|
|  \$10,000   |       58%  |
|  \$25,000   |      64%  |
|  \$50,000   |      68%  |
|  \$75,000   |      54%  |
|  \$100,000  |      54%  |
|  \$125,000  |       46%  |
|  \$150,000  |       55%  |
|  \$175,000  |       58%  |
|  \$200,000  |       50%  |
|  All        |       58%  |

#### Without performing Statistical Analysis on the Data, it is hard to say if these numbers are significant.  
* On average, 58% of people travelled during thanksgiving. 
* On average, the $10k, $25k, and $50k income grous travelled 58%, 64%, and 68% of the time respectively.  
* All of these buckets were above average.  
* Again - Can't yet tell if these numbers are significant. 

## Calculate breakdown of travel for age group
#### Let's look at the same question, but for age group.  

In [17]:
travel_by_age = pd.pivot_table(americans,
                               values=[index_id],
                               index=[travel_q], 
                               columns=["int_age"],
                               aggfunc=len,
                               margins=True)




In [18]:
print(travel_by_age)

                                                   RespondentID                \
int_age                                                    18.0   30.0   45.0   
How far will you travel for Thanksgiving?                                       
Thanksgiving is happening at my home--I won't t...         59.0   90.0  122.0   
Thanksgiving is local--it will take place in th...         55.0   77.0   71.0   
Thanksgiving is out of town and far away--I hav...         15.0   20.0   31.0   
Thanksgiving is out of town but not too far--it...         56.0   48.0   45.0   
All                                                       185.0  235.0  269.0   

                                                                  
int_age                                              60.0    All  
How far will you travel for Thanksgiving?                         
Thanksgiving is happening at my home--I won't t...  123.0  394.0  
Thanksgiving is local--it will take place in th...   72.0  275.0  
Thanksgiving is 

# Linking Friendship with Age and Income

In [19]:
hometown_q = "Have you ever tried to meet up with hometown friends on Thanksgiving night?"
friendsgiving_q = 'Have you ever attended a "Friendsgiving?"'

friendship_age = americans.pivot_table(index=hometown_q, 
                                       columns=[friendsgiving_q], 
                                       values="int_age")

print(friendship_age)

Have you ever attended a "Friendsgiving?"                  No        Yes
Have you ever tried to meet up with hometown fr...                      
No                                                  42.283702  37.010526
Yes                                                 41.475410  33.976744


In [20]:
friendship_income = americans.pivot_table(index=hometown_q, 
                                          columns=[friendsgiving_q], 
                                          values="int_income")
print(friendship_income)

Have you ever attended a "Friendsgiving?"                     No           Yes
Have you ever tried to meet up with hometown fr...                            
No                                                  78914.549654  72894.736842
Yes                                                 78750.000000  66019.736842
